In [ ]:
!pip install python-dotenv transformers datasets==2.10.0 accelerate wandb git+https://github.com/huggingface/diffusers

In [ ]:
from dotenv import load_dotenv
import os


load_dotenv()
read_token = os.environ["READ_HF_TOKEN"]
write_token = os.environ["WRITE_HF_TOKEN"]
os.system(f"!huggingface-cli login --token {write_token}")

### Гиперпараметры

In [ ]:
base_model_path = "stabilityai/stable-diffusion-2-1-base"
dataset_path = "logo-wizard/modern-logo-dataset"
image_resolution = 512
batch_size = 16
grad_accum_steps = 1
train_steps = 10000
lr = 1e-4
scheduler = "cosine"
output_dir = "/lora"
final_lora_name = "logo-diffusion-lora"
checpointing_steps = 500
val_prompt = "Logo of bakery with pretzel."

### Запуск обучения

In [ ]:
os.system(
    f"accelerate launch --mixed_precision='fp16'  lora_finetune_script.py \
  --pretrained_model_name_or_path={base_model_path} \
  --dataset_name={dataset_path} \
  --dataloader_num_workers=8 \
  --resolution={image_resolution} --center_crop --random_flip \
  --train_batch_size={batch_size} \
  --gradient_accumulation_steps={grad_accum_steps} \
  --max_train_steps={train_steps} \
  --learning_rate={lr} \
  --max_grad_norm=1 \
  --lr_scheduler={scheduler} --lr_warmup_steps=0 \
  --output_dir={output_dir} \
  --push_to_hub \
  --hub_model_id={final_lora_name} \
  --report_to=wandb \
  --checkpointing_steps={checpointing_steps} \
  --validation_prompt={val_prompt} \
  --enable_xformers_memory_efficient_attention \
  --seed=42"
)